The python code mainly consists of the machine learning training and prediction parts of the project.

In [3]:
#导入需要的包
#load packages
import pyreadr
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import lightgbm as lgb
from skopt import BayesSearchCV
from xgboost import XGBClassifier
import time

In [5]:
#读取数据
# read the data
data1 = pyreadr.read_r("E:\\qjy\\ecnu\\大三第二学期\\生物医疗\\生物医疗小组作业1\\CD.Rdata")
X_train = data1["X_train"]#feature
X_test = data1["X_test"]
y_train = data1["y_train"]#target
#是否有缺失值
# There is no missing value.
sum(pd.isnull(X_train).any())
sum(pd.isnull(X_test ).any())
sum(pd.isnull(y_train).any())

0

In [6]:
#读取基于BH方法变量筛选方法处理后的X_train、X_test数据
# Read the data X_train and X_test processed by the variable screening method based on BH method
X_BH= pd.read_csv("E:\\qjy\\ecnu\\大三第二学期\\生物医疗\\生物医疗小组作业1\\X_BH.csv")
X_test_BH = pd.read_csv("E:\\qjy\\ecnu\\大三第二学期\\生物医疗\\生物医疗小组作业1\\X_test_BH.csv")

In [7]:
X_BH= pd.read_csv("E:\\qjy\\ecnu\\大三第二学期\\生物医疗\\生物医疗小组作业1\\X_BH.csv")
y_train = pd.read_csv("E:\\qjy\\ecnu\\大三第二学期\\生物医疗\\生物医疗小组作业1\\y_train.csv")

# 基本分类模型和集成学习 Basic Classification Models and Embedding Models

## 利用网格搜索进行初步探索  An initial exploration was performed using grid search

In [16]:
# Initialize AUC list
auc_scores = []

# Run the loop 50 times
for i in range(1, 51):
    # Split genotype data and phenotype data into training and testing sets
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)
    
    # KNN model
    k_values = list(range(20, 31))  # Set the candidate range for K values
    auc_score_knn = []   # To save cross-validation AUC results for each K value
    for k in k_values:
        clf = KNeighborsClassifier(n_neighbors=k, p=1, metric='minkowski')
        auc = np.mean(cross_val_score(clf, X_train1, y_train1, cv=5, scoring="roc_auc"))
        auc_score_knn.append(auc)  # Perform cross-validation on x_train and calculate AUC for each K value
    best_k = k_values[np.argmax(auc_score_knn)]  # Find the best performing K value
    model_knn = KNeighborsClassifier(n_neighbors=best_k, p=1, metric='minkowski')  # Create KNN model
    model_knn.fit(X_train1, y_train1)   # Train the model on the entire x_train
    y_pred = model_knn.predict(X_test1)  # Use the trained model to predict the test data
    auc_scores.append(roc_auc_score(y_test1, y_pred))   # Calculate AUC
    
    # Logistic regression
    lr = LogisticRegression(random_state=i)
    lr_param_grid = {'C': [0.1, 1.0, 10.0]}
    lr_grid_search = GridSearchCV(lr, lr_param_grid, scoring='roc_auc', cv=5)
    lr_grid_search.fit(X_train1, y_train1)
    lr_best_model = lr_grid_search.best_estimator_
    lr_auc = roc_auc_score(y_test1, lr_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(lr_auc)
    
    # Decision Trees
    param = [{'max_depth': np.arange(3, 20, 3), 'min_samples_leaf': np.arange(6, 10, 2),
              'criterion': ['gini', 'entropy']}]
    tlf = GridSearchCV(DecisionTreeClassifier(random_state=i), param_grid=param, cv=5)  
    # Use grid search on x_train to select the best parameters (which includes cross-validation)
    tlf.fit(X_train1, y_train1)
    model_tree = DecisionTreeClassifier(criterion=tlf.best_params_["criterion"],
                                        max_depth=tlf.best_params_["max_depth"],
                               min_samples_leaf=tlf.best_params_["min_samples_leaf"])
    # Build decision tree classification model using the locally best-performing parameter values
    model_tree.fit(X_train1, y_train1)
    y_pred = model_tree.predict_proba(X_test1)[:, 1]
    auc_scores.append(roc_auc_score(y_test1, y_pred))
    
    # Random forest model
    rf = RandomForestClassifier(random_state=i)
    rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [1, 5, 10]}
    rf_grid_search = GridSearchCV(rf, rf_param_grid, scoring='roc_auc', cv=5)
    rf_grid_search.fit(X_train1, y_train1)
    rf_best_model = rf_grid_search.best_estimator_
    rf_auc = roc_auc_score(y_test1, rf_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(rf_auc)

    # GBDT model
    gbdt = GradientBoostingClassifier(random_state=i)
    gbdt_param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    gbdt_grid_search = GridSearchCV(gbdt, gbdt_param_grid, scoring='roc_auc', cv=5)
    gbdt_grid_search.fit(X_train1, y_train1)
    gbdt_best_model = gbdt_grid_search.best_estimator_
    gbdt_auc = roc_auc_score(y_test1, gbdt_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(gbdt_auc)

    # SVM model
    svm = SVC(probability=True, random_state=i)
    svm_param_grid = {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']}
    svm_grid_search = GridSearchCV(svm, svm_param_grid, scoring='roc_auc', cv=5)
    svm_grid_search.fit(X_train1, y_train1)
    svm_best_model = svm_grid_search.best_estimator_
    svm_auc = roc_auc_score(y_test1, svm_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(svm_auc)

    # LightGBM model
    lgbm = lgb.LGBMClassifier(random_state=i)
    gbm_param_grid = {'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    gbm_grid_search = GridSearchCV(lgbm, gbm_param_grid, scoring='roc_auc', cv=5)
    gbm_grid_search.fit(X_train1, y_train1)
    gbm_best_model = gbm_grid_search.best_estimator_
    gbm_auc = roc_auc_score(y_test1, gbm_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(gbm_auc)
    
    # XGBoost
    xgb = XGBClassifier(random_state=i)
    xgb_param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1, 0.3],
        'min_child_weight': [3, 4, 5, 6]
    }
    xgb_grid_search = GridSearchCV(xgb, xgb_param_grid, scoring='roc_auc', cv=5)
    xgb_grid_search.fit(X_train1, y_train1)
    xgb_best_model = xgb_grid_search.best_estimator_
    xgb_auc = roc_auc_score(y_test1, xgb_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(xgb_auc)
    print("This is iteration", i)

# Put the data into a table
auc_scores = pd.DataFrame(data=auc_scores)
auc_scores.to_csv('auc_scores.csv')

这是循环 1


## 基于贝叶斯优化进一步调参优化  Bayesian optimization is used to further tune the hyparameters

以下代码由于运算时间问题所以没有写在一个for循环下面，但是通过设置相同的随机种子来控制对于不同方法的训练集和测试集相同

The following code is not written under a for loop due to time constraints, but sets the same random seed to ensure that the training and test sets are the same for each method.

In [ ]:
# LightGBM loop for 50 iterations
n_iterations = 50
auc_scores_lightGBM_BH = []
time_lightGBM_BH = []

for i in range(n_iterations):
    
    start_time = time.time()
    
    # Split the training set and testing set
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)  # Random seed set

    # Define LightGBM classifier
    lgb_classifier = lgb.LGBMClassifier(random_state=i)

    # Define parameter space
    param_space = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'num_leaves': (10, 100),
        'min_child_samples': (1, 20),
        'subsample': (0.1, 1.0, 'uniform'),
        'colsample_bytree': (0.1, 1.0, 'uniform')
    }

    # Use Bayesian optimization for hyperparameter tuning
    optimizer = BayesSearchCV(lgb_classifier, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # Output the best parameters
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # Make predictions on the test set
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # Calculate AUC score
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_lightGBM_BH.append(auc)
    #print(f"Iteration {i+1}: AUC - {auc}")
    
    end_time = time.time()
    duration = end_time - start_time
    time_lightGBM_BH.append(duration)
    
    
# Put the AUC scores into a table
auc_scores_lightGBM_BH = pd.DataFrame(data=auc_scores_lightGBM_BH)
auc_scores_lightGBM_BH.to_csv('auc_scores_lightGBM_BH.csv')

# Put the time durations into a table
time_lightGBM_BH = pd.DataFrame(data=time_lightGBM_BH)
time_lightGBM_BH.to_csv('auc_scores_lightGBM_BH.csv')

In [ ]:
# SVM loop for 50 iterations
n_iterations = 50
auc_scores_SVM_BH = []
time_SVM_BH = []

for i in range(n_iterations):
        
    start_time = time.time()
    
    # Split the training set and testing set
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)  # Random seed set

    # Define SVM classifier
    svm_classifier = SVC(probability=True, random_state=i)

    # Define parameter space
    param_space = {
        'C': (0.01, 10.0, 'log-uniform'),
        'gamma': (0.01, 10.0, 'log-uniform'),
        'kernel': ('linear', 'rbf')
    }

    # Use Bayesian optimization for hyperparameter tuning
    optimizer = BayesSearchCV(svm_classifier, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # Output the best parameters
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # Make predictions on the test set
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # Calculate AUC score
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_SVM_BH.append(auc)
    print(f"Iteration {i+1}: AUC - {auc}")
        
    end_time = time.time()
    duration = end_time - start_time
    time_SVM_BH.append(duration)

# Put the AUC scores into a table
auc_scores_SVM_BH = pd.DataFrame(data=auc_scores_SVM_BH)
auc_scores_SVM_BH.to_csv('auc_scores_SVM_BH.csv')

# Put the time durations into a table
time_SVM_BH = pd.DataFrame(data=time_SVM_BH)
time_SVM_BH.to_csv('time_SVM_BH.csv')

In [ ]:
# Random Forest loop for 50 iterations
n_iterations = 50
auc_scores_RF_BH = []
time_RF_BH = []

for i in range(n_iterations):
        
    start_time = time.time()
    
    # Split the training set and testing set
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)  # Random seed set

    # Define Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=i)

    # Define parameter space
    param_space = {
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'max_features': (0.1, 1.0, 'uniform')
    }

    # Use Bayesian optimization for hyperparameter tuning
    optimizer = BayesSearchCV(rf_classifier, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # Output the best parameters
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # Make predictions on the test set
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # Calculate AUC score
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_RF_BH.append(auc)
    print(f"Iteration {i+1}: AUC - {auc}")
        
    end_time = time.time()
    duration = end_time - start_time
    time_RF_BH.append(duration)

# Put the AUC scores into a table
auc_scores_RF_BH = pd.DataFrame(data=auc_scores_RF_BH)
auc_scores_RF_BH.to_csv('auc_scores_RF_BH.csv')

# Put the time durations into a table
time_RF_BH = pd.DataFrame(data=time_RF_BH)
time_RF_BH.to_csv('time_RF_BH.csv')

In [ ]:
# GBDT loop for 50 iterations
n_iterations = 50
auc_scores_BH_GBDT = []
time_BH_GBDT = []

# Define parameter space
param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (50, 500),
    'max_depth': (1, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'max_features': (0.1, 1.0, 'uniform')
}

for i in range(n_iterations):
        
    start_time = time.time()
    
    # Split the training set and testing set
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)  # Random seed set
    
    # Define GBDT classifier
    gbdt = GradientBoostingClassifier(random_state=i)
    
    # Use Bayesian optimization for hyperparameter tuning
    optimizer = BayesSearchCV(gbdt, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train2, y_train2)
    
    # Output the best parameters
    print(f"Best params: {optimizer.best_params_}")
    
    # Make predictions on the test set
    y_pred = optimizer.predict_proba(X_test2)[:, 1]
    
    # Calculate AUC score
    auc = roc_auc_score(y_test2, y_pred)
    auc_scores_BH_GBDT.append(auc)
    
    #print("This is iteration", i)
        
    end_time = time.time()
    duration = end_time - start_time
    time_BH_GBDT.append(duration)
    
# Put the AUC scores into a table
auc_scores_BH_GBDT = pd.DataFrame(data=auc_scores_BH_GBDT)
auc_scores_BH_GBDT.to_csv('auc_scores_BH_GBDT.csv')

# Put the time durations into a table
time_BH_GBDT = pd.DataFrame(data=time_BH_GBDT)
time_BH_GBDT.to_csv('time_BH_GBDT.csv')

# 基于GBDT进行预测

In [8]:
# Define parameter space
param_space = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'num_leaves': (10, 100),
        'min_child_samples': (1, 20),
        'subsample': (0.1, 1.0, 'uniform'),
        'colsample_bytree': (0.1, 1.0, 'uniform')
}

# Define GBDT classifier
lgb = lgb.LGBMClassifier(random_state=42)

# Use Bayesian optimization for hyperparameter tuning
optimizer = BayesSearchCV(lgb, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
optimizer.fit(X_BH, y_train)

# Output the best parameters
print(f"Best params: {optimizer.best_params_}")

# Make predictions on the test set
y_pred = optimizer.predict_proba(X_test_BH)[:, 1]

# Put predictions into a table
y_pred_BH_lightGBT = pd.DataFrame(data=y_pred)
y_pred_BH_lightGBT.to_csv('y_pred_BH_lightGBT.csv')

[LightGBM] [Info] Number of positive: 1493, number of negative: 2588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002227 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 4081, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.365842 -> initscore=-0.550098
[LightGBM] [Info] Start training from score -0.550098
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b